# 네이버 블로그 크롤링 (네이버 API 이용)


## 들어가며...



## 1) 크롤링? 파싱?

### 1-1) 크롤링

- '웹 크롤러'라는 단어에서 유래되었음.
- 크롤러란 조직적, 자동화된 방법으로 WWW를 탐색하는 컴퓨터 프로그램.
- 크롤링은 크롤러가 하는 작업을 부르는 말.
- 여러 인터넷 사이트의 페이지(문서,html 등)를 수집해서 분류하는 것.
- 대체로 찾아낸 데이터를 저장한 후 쉽게 찾을 수 있게 인덱싱 수행.

### 1-2) 파싱

- 파싱이란 어떤 페이지(문서, html 등)에서 내가 원하는 데이터를 특정 패턴이나 순서로 추출하여 정보를 가공하는 것.
- 파싱이란 일련의 문자열을 의미있는 '토큰'으로 분해하고 이들로 이루어진 '파스 트리'를 만드는 과정.
- 입력 토큰에 내제된 자료 구조를 빌드하고 문법을 검사하는 역할을 함.

## 2) 애플리케이션 등록 ( API 이용신청)

![애플리케이션 등록_1](https://user-images.githubusercontent.com/51112316/61574953-2f655d80-ab01-11e9-8b8f-cc74183302da.jpg)
![애플리케이션 등록_2](https://user-images.githubusercontent.com/51112316/61574954-31c7b780-ab01-11e9-9274-e1d237547d94.jpg)

### 2-1) 네이버API  예제 사용

- 네이버개발자 -> Products -> 서비스API -> 검색 -> 개발 가이드 보기 -> 0.API호출예제 -> Python

#### [API호출예제](https://developers.naver.com/docs/search/blog/)

~~~python
import java.io.BufferedReader;
import java.io.InputStreamReader;
import java.net.HttpURLConnection;
import java.net.URL;
import java.net.URLEncoder;

public class APIExamSearchBlog {

    public static void main(String[] args) {
        String clientId = "YOUR_CLIENT_ID";//애플리케이션 클라이언트 아이디값";
        String clientSecret = "YOUR_CLIENT_SECRET";//애플리케이션 클라이언트 시크릿값";
        try {
            String text = URLEncoder.encode("그린팩토리", "UTF-8");
            String apiURL = "https://openapi.naver.com/v1/search/blog?query="+ text; // json 결과
            //String apiURL = "https://openapi.naver.com/v1/search/blog.xml?query="+ text; // xml 결과
            URL url = new URL(apiURL);
            HttpURLConnection con = (HttpURLConnection)url.openConnection();
            con.setRequestMethod("GET");
            con.setRequestProperty("X-Naver-Client-Id", clientId);
            con.setRequestProperty("X-Naver-Client-Secret", clientSecret);
            int responseCode = con.getResponseCode();
            BufferedReader br;
            if(responseCode==200) { // 정상 호출
                br = new BufferedReader(new InputStreamReader(con.getInputStream()));
            } else {  // 에러 발생
                br = new BufferedReader(new InputStreamReader(con.getErrorStream()));
            }
            String inputLine;
            StringBuffer response = new StringBuffer();
            while ((inputLine = br.readLine()) != null) {
                response.append(inputLine);
            }
            br.close();
            System.out.println(response.toString());
        } catch (Exception e) {
            System.out.println(e);
        }
    }
}
~~~

#### 문제 발생 :  

#### 네이버 블로그 형식 맞춰 긁어왔음.
                  
#### 사실은 요약된 내용.

#### 모든 내용을 다 보고싶으면 개발이 필요.

# 네이버 블로그 크롤링

In [1]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup

naver_Client_id = "E4zDnlSmVAnJ7dvIOUUx"
naver_Client_secret = "lLxEw7DWbB"

In [2]:
def get_blog_count(query,display) :
    encode_query=urllib.parse.quote(query)
    search_url="https://openapi.naver.com/v1/search/blog?query="+ encode_query
    # 위의 https://~ 는 따온 API호출예제에 있음.
    request=urllib.request.Request(search_url)
    # 이렇게 하면 내가 만들었던 url 형태로 요청한다.
    # 그러나 이렇게 하면 API로부터 거부당한다.
    # 왜냐하면 ID, Secret을 주지 않았기 때문에 아래와 같이 이것을 입력해준다.
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    # code를 받아와서 웹에서 정상적으로 접속했을 때 200번을 준다.
    if response_code is 200 :
        response_body = response.read()  # 실제로 읽어 들인다.
        response_body_dict = json.loads(response_body.decode('utf-8'))
        # 네이버에서 넘겨줄 때 json으로 넘겨주는데 이것을 전부 '딕셔너리'로 저장한다.
        
        print('Last build date :' + str(response_body_dict['lastBuildDate']))
        # json으로 넘겨받을 것들 중에서 최종날짜를 출력
        print('Total :' + str(response_body_dict['total']))  # 관측개수
        print('Start :' + str(response_body_dict['start']))
        print('Display :' + str(response_body_dict['display']))
        #이렇게 넘어온 것을 다 출력해본다.
        
        # 검색된 결과가 0 일 경우        
        if response_body_dict['total']== 0:
            blog_count = 0
        else :
            blog_total=math.ceil(response_body_dict['total']/int(display))
            # 위에서 한 번에 10개씩 출력한다고 했으니 나눠주어서 개수를 맞춰준다.
        
            if blog_total >= 1000 :
                blog_count = 1000
                # 최대 1000개로 제한이 있으니 1000개를 넘었을 때는 1000개로 지정한다.
            else : 
                blog_count = blog_total
        
            print('Blog total :' + str(blog_total))
            print('Blog count : ' + str(blog_count))
        
        return blog_count

- 블로그의 모든 내용을 가져오는 함수[(API사용예제)는 샘플 내용만 가져온다.]



In [3]:
def get_blog_post(query,display,start_index,sort):
    global no, fs
    #get_blog_count(query,display)를 정의할 때 사용했던 것과 동일하게 사용(아래 7문장)
    encode_query = urllib.parse.quote(query)
    # 동일하게 사용하나 search_url에는 옵션을 조금 더 추가해 준다.
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query +"&display=" + str(display) + "&start=" + str(start_index) + "sort=" + sort
    request=urllib.request.Request(search_url)
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    #print(response_code) 이분은 윗부분까지 해서 접속이 잘되었는지를 확인하는 것.
        
    if response_code is 200:
        response_body=response.read()
        response_body_dict=json.loads(response_body.decode('utf-8'))
        # json이 해당하는 블로그가 여러개 나오면 전부 다 하나의 큰 덩어리로 주는데
        # 나눠서 하나씩 접근 하기 위해 아래의 코드를 짜줌.
        for item_index in range(0,len(response_body_dict['items'])):
            # 혹시 태그부분이 따라올 수 있기 때문에
            # 그것을 지우고 사용하기 위해  re(regular expression)으로 제거를 해준다.
            # 즉 태그를 없애주는 정규표현식을 이용한다.
            try :
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['title'])
                #또한 주소가 따라오는데 이건 필요 없는 것이다.그러므로 없애준다.
                link = response_body_dict['items'][item_index]['link'].replace("amp;","")
                # description에도 마찬가지로 태그가 따라올 수 있기 때문에 제거해준다.
                description = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                #  포스트(개시글)의 주소가 아니라 블로그 자체의 링크를 가져옴.
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']
                #아래처럼 변환해서 가져와도 되고 그냥 주는대로 가져와도 된다.
                #post_date = datetime.datetime.strptime(response_body_dict[['item'][item_index]['postdate'],"%Y%m%d").strtime("%Y.%m.%d")
                
               # 이제 json에서 오는 결과를 하나씩 items에 다 들어가 있다. 그것을 이제 다 끄집어서 가져온다.
                no +=1
                print("-------------------------------------------------") #각각의 게시글을 구분시킴
                print("#"+ str(no))  #몇번째 게시글인지 출력
                print("Title :"+ title)
                print("Link :"+ link)
                print("Description :"+ description)
                print("Blogger Name :"+ blogger_name)
                print("Blogger Link :"+ blogger_link)
                print("Post Date :"+ post_date)
                
                                                                              
                post_code = requests.get(link)
                #  저 정보들은 사실 네이버 오픈 API에서 제공해주는 정보다.
                # 그런데 우리가 필요한 것은 실제 블로그에 포스트 된 내용이 필요하기 때문에
                # 뷰티풀숲을 이용해 가져온다. 
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text,'lxml')
                
                # 네이버 블로그의 문제점 : 마우스를 긁어오지 못하게 iframe으로 만들어 놓은게 있다.
                #그래서 iframe#mainFrame 이라는 html 부분을 들어가서
                # 그 부분의 실제 post url을 뽑을 수 있다.
                # 그냥 가져와선 안되고 그 url에서 소스부분(src)만 가져온다.
                # 그래서 그걸로 접속해서 실제적인 블로그 포스트에 대한 내용을 긁어오기 때문에
                # 여기에 다시 한번 "blog_post_url"이라는 주소로 접근을 한다.
                # 그리고 그것들에 대한 정보를 가져온다.
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com"+mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text,'lxml')
                    
                    # 블로그 전체의 여러가지 메뉴들도 있고 그런 것 제외하고 postViewArea라는 부분에
                    # 실제 포스트 내용이 들어있다. 그래서 여기로 선택한다. 이게 진짜 블로그 포스트의 컨텐트다. 
                    for blog_post_content in blog_post_soup.select("div#postViewArea"):
                        blog_post_content_text = blog_post_content.get_text()
                        blog_post_full_contents = str(blog_post_content_text) #컨텐트를 string으로 캐스팅해서 가져온다.
                        blog_post_full_contents = blog_post_full_contents.replace("\n\n",'\n') 
                        #요약된 description 만 주는게 아니라 이젠 진짜로 된 포스트를 준다.
                        #print("blog_post_contents : " + blog_post_full_contents+"\n")
                        #전체 내용을 print 하면 너무 많기 때문에 위의 코드로 하지않고 파일로 저장한다.
                        #이를 통해 이후에 데이터 분석 등에 사용한다.
                        fs.write(blog_post_full_contents+'\n')
                        fs.write("-----------------------------------")
            #만약에 못할 경우 그냥 넘어가는 코드.
            except: 
                item_index+=1
                print("■■■■■■저장할 수 없음.■■■■■■")

### - 아래 모든 코드를 자동화시키기 위해 query="키워드"로 한 후에 shift+ENTER만 갈긴다.

In [4]:
query="박대"

In [6]:
if __name__=='__main__' :
    no = 0           # 몇 개의 포스트를 저장했는지 세기 위한 index
    #query = "대구" # 검색을 원하는 문자열로써 UTF-8로 인코딩한다.
    display = 10     # 검색 결과 출력 건수 지정, 10(기본값), 100(최대값)
    start = 1        # 검색 시작 위치로 최대 1000까지 가능
    sort = 'date'    # 정렬 옵션 : sim(유사도순, 기본값), date(날짜순)
    
    #블로그 콘텐츠의 한글 저장을 위해 encoding='utf-8'으로 설정.
    fs = open(query + ".txt",'a',encoding='utf-8')
    
    blog_count = get_blog_count(query,display)
    for start_index in range(start,blog_count + 1, display):
        get_blog_post(query,display,start_index,sort)  
    
       
    fs.close()

Last build date :Fri, 26 Jul 2019 14:26:42 +0900
Total :44039
Start :1
Display :10
Blog total :4404
Blog count : 1000
-------------------------------------------------
#1
Title :#133. 박대(박대기)/각시서대/노랑각시서대/궁제기서대
Link :https://blog.naver.com/gy57927?Redirect=Log&logNo=221510831367
Description :박대 (카라아카시타비라메 カラアカシタビラメ) 각시서대 (세토우시노시타 セトウシノシタ)... 박대만 대가리가 좌측으로 향해 있고, 나머지 어종들은 대가리가 우측을 향하고 있다. 이게 바로 참서대과와... 
Blogger Name :미스터S 의 IN&OUT (indoor&outdoor)
Blogger Link :https://blog.naver.com/gy57927
Post Date :20190411
-------------------------------------------------
#2
Title :군산 박대를 처음으로 사본다.
Link :https://blog.naver.com/mmmagic?Redirect=Log&logNo=221593577141
Description :#군산박대 늦은 시간 애를 재우고 글을 한개 씁니다. 어제 처음으로 군산에서 돈을 주고 박대를 사봤습니다. 저는 평생 제 돈 주고 군산에서 박대를 사본적이 없습니다. 그 이유는... http://www.obs.co.kr/new/program... 
Blogger Name :팡리티!!?
Blogger Link :https://blog.naver.com/mmmagic
Post Date :20190723
-------------------------------------------------
#3
Title :나물무침, 고추장주물럭, 명품전, 돼지고지짜글이, 박대구이
Link :h

-------------------------------------------------
#23
Title :박대구이 박대 반찬 밥한그릇
Link :https://blog.naver.com/mohangrim?Redirect=Log&logNo=221463641471
Description :안중근이 말하길 하루라도 책을 읽지 읽으면 입에 가시가 돋는다 했다는데 그냥 넘기기엔 너무나 밍밍해 서해에서 많이 나는 해산물 중에 서대라 부르기도하는 박대를 들고 나왔내요. 누구나 좋아하기에 박대인... 
Blogger Name :아름다운 세상 이야기
Blogger Link :https://blog.naver.com/mohangrim
Post Date :20190212
-------------------------------------------------
#24
Title :군침도는 최고수산 반건조생선 군산박대 김포대명항 쉬운요리... 
Link :https://blog.naver.com/puresprite?Redirect=Log&logNo=221526965510
Description :반건조 생선 군산박대 최고수산 국내산 최고품질 고집 굽는 걸로 손쉽게 하면서도 진미요리가 됩니다. 최고수산 군산박대는 과연 어떤 맛일까요? 군산박대 국내산 최고수산 모방송사에서 김수미 수미반찬... 
Blogger Name :달콤 추억이 방울방울
Blogger Link :https://blog.naver.com/puresprite
Post Date :20190501
-------------------------------------------------
#25
Title :청암수산 박대세트로 맛있는 박대조림 어떠세요?
Link :https://blog.naver.com/mag01?Redirect=Log&logNo=221267825464
Description :즐거운 연휴에 빠질 수 없는게 바로 음식인데요 박대조림 어떠세요? 사실 저도 박대는 처음 접해봤는데 이거 완전 매력있네요. 조려주니 아

-------------------------------------------------
#46
Title :5살.아이가 잘먹는 고소한 박대구이.아이식단.
Link :https://blog.naver.com/jensksj?Redirect=Log&logNo=221576490038
Description :5살.아이가 잘먹는 고소한 박대구이.아이식단. 5살 아이가 맛있다며 잘먹는. 냄새없는 쫄깃쫄깃 꼬소한 박대구이. 박대 아시나요? 지역에 따라 박대 또는 서대라고 하는데. 어릴적 친정아빠가 박대를 좋아하셔서.... 
Blogger Name :랄프로렝 sTory
Blogger Link :https://blog.naver.com/jensksj
Post Date :20190703
-------------------------------------------------
#47
Title :아리랑 박대정식 군산근대역사박물관
Link :https://blog.naver.com/sumer_sky?Redirect=Log&logNo=221580184188
Description :보쌈~~ 박대랑 전복장 잡채랑 도토리묵도 맛있었는데 못찍~~ 인당 22천원~~ 살짝 비싼감도 있지만~~~ 가족단위에겐 딱 좋은 !! #군산박대정식 #군산아리랑 #군산맛집 그리고 박물관투어~~권역통합권 3천원인데... 
Blogger Name :SUMMER
Blogger Link :https://blog.naver.com/sumer_sky
Post Date :20190708
-------------------------------------------------
#48
Title :아리울수산 군산박대 반건조 생선 그 맛 아실랑가~~?
Link :https://blog.naver.com/joogy?Redirect=Log&logNo=221503903107
Description :먹어본 박대라는 생선 살짝 반건조 된 가자미처럼 얇은 생선을 먹어봤어요. 이거 구워도 찜으로도 조림으로도 너무 맛있고 또 귀한 생선이더라

-------------------------------------------------
#68
Title :군산 박대를 다양하게 먹을 수 있는 새만금횟집
Link :https://blog.naver.com/princesshje?Redirect=Log&logNo=221472141171
Description :그래서 이것저것 검색해보니 #군산박대 가 그렇게 유명하더라고요 티비 프로그램에서도 군산 박대가 나왔었거든요 서대는 들어봤는데 박대가 뭐지? 검색을 해봤어요 서해안 특산품인 생선 #박대 는 바다 속... 
Blogger Name :매력이의 소소한 이야기
Blogger Link :https://blog.naver.com/princesshje
Post Date :20190222
-------------------------------------------------
#69
Title :[전북,군산,신창동/구이나라] 박대구이가 땡겨서 갔는데 박대조... 
Link :https://blog.naver.com/hdhdd?Redirect=Log&logNo=221373899376
Description :사실 생선구이중 박대구이를 원츄했기에 폭풍검색을 했습니다. 별다른곳이 떠오르지 않아서 그냥 숙소앞에 좋아보이는곳으로 결정했습니다. 골목 바로 앞에 위치해 있더라구요 메뉴판 박대구이를 먹어줄까... 
Blogger Name :책과 음식과 영화이야기
Blogger Link :https://blog.naver.com/hdhdd
Post Date :20181029
-------------------------------------------------
#70
Title :생선조림 쫀득하고 맛있는 박대 조림
Link :https://blog.naver.com/wjdtj54?Redirect=Log&logNo=221254407221
Description :생선 박대를 아시나요? 박대라고도 하고 서대라고도 하는 맛있는 생선이랍니다 꾸들꾸들 말린 박대를 조림이나 구웠을 때 살이 똑똑 떨어

-------------------------------------------------
#90
Title :&lt;생선굽는고양이&gt; 서대 박대
Link :https://blog.naver.com/sugi_728?Redirect=Log&logNo=221410803262
Description :박대? 둘 다 같은 생선인데 박대는 서대의 썹질을 벗긴 것이라고 하네요. 껍질이 있는 서대가 식감도 더 쫀득하다고 합니다. 참고해주세요~ 저는 서대, 박대를 블로그 지인을 통해 처음 알았어요. 생선이... 
Blogger Name :토리 n 찐빵
Blogger Link :https://blog.naver.com/sugi_728
Post Date :20181202
-------------------------------------------------
#91
Title :[수지 동천동 맛집] 소주한잔하기 좋은 《박대와막창》
Link :https://blog.naver.com/8282kimjh?Redirect=Log&logNo=221416379634
Description :수지 동천동 막창 맛집 &lt;박대와 막창&gt; 바로 옆 맥주생각도 맛있음ㅎㅎ 맛도 있고, 푸짐하고, 항상 친절하신 사장님이 계신 동천동 박대와 막창^^ 막창2인분 주문 사장님께서 미리 초벌해서 가져다 주신다... 
Blogger Name :코비의 맛집 여행~
Blogger Link :https://blog.naver.com/8282kimjh
Post Date :20181210
-------------------------------------------------
#92
Title :군산박대 꾸이 바삭한게 아이들 간식으로 굿!
Link :https://blog.naver.com/flyingchichi?Redirect=Log&logNo=221408381186
Description :요즘 핫한 군산박대가 들어간 스낵이 있어 준비해봤어요. 박대는 군산에서 많이 잡히는 생선인데요. 비린내가 안나고 육질이 부

...

-------------------------------------------------
#897
Title :반건조로 꼬들함이 살아있는 서대(박대) 아이들 반찬으로 최고 :)
Link :https://blog.naver.com/saraya0802?Redirect=Log&logNo=221407017771
Description :서대(박대)인데요 수미네 반찬에 소개되어 많은 분들이 찾는다는 박대는 서대의 껍질을 벗긴 것이랍니다 생선껍질에 피쉬콜라겐이 듬뿍 들어있고, 체내 흡수율도 좋습니다 식감도 쫀득하니 정말 맛이 좋답니다... 
Blogger Name :꽃청춘 엄마 사라댁
Blogger Link :https://blog.naver.com/saraya0802
Post Date :20181127
-------------------------------------------------
#898
Title :서천특화시장에서 전어,회먹고 꽃게,박대 사오고~
Link :https://blog.naver.com/oticonhappy?Redirect=Log&logNo=221372098422
Description :꽃게, 대하, 박대 를 저희 3남매 집에 골고루 사주셨어요^^ 저는 박대라는 생선은 처음 봤네요^^ ㅎ 많이 샀는데 카드결제가 안된다는 말씀에.. 허거걱~~ 하고 다시 흥정시작.. ㅎ 꽃게 한마리 더 주셨어요.. ㅎ 얘가... 
Blogger Name :오티콘보청기 광명센터 / 진심을 담아
Blogger Link :https://blog.naver.com/oticonhappy
Post Date :20181006
-------------------------------------------------
#899
Title :마마무 한끼줍쇼 솔라,화사 장안동에서 한끼 성공 곱창,박대
Link :https://blog.naver.com/chopa1394?Redirect=Log&logNo=221326662326
Description :180725 마마무 한끼줍쇼 솔라,화사 

-------------------------------------------------
#919
Title :김치콩나물국 해장 &amp; 군산 박대 구이
Link :https://blog.naver.com/yjoshjh?Redirect=Log&logNo=110182047831
Description :오랜만에 김치콩나물국 해장 집에서 먹는 해장국중엔 단연 최고죠 바지락살을 넣으니 더 좋군요 ^^ 밥 말아서 훌훌 ~~ 통통하고 큼지막한 군산 박대 구이 옆에 낑겨있는 찌깐한 굴비 구이는 쨉도 안됩니다 ^^
Blogger Name :참이술의 Eat & Drink
Blogger Link :https://blog.naver.com/yjoshjh
Post Date :20131225
-------------------------------------------------
#920
Title :원양산 홍서대 판매, 선별포장 참박대와 가성비 비교 보너스
Link :https://blog.naver.com/newhasu1?Redirect=Log&logNo=221467933790
Description :원양산 홍서대(기니개서대) 판매명: 서대(박대) 선별 끝 포장시작~ 그전에 국산 참박대(박대)와 같은 가격 크기비교 보너스 샷 국산 참박대 110g과 원양산홍서대 200~250g 같은 마진률 홍서대가 좀 더 싼 가격... 
Blogger Name :군산건어물(행남수산) 도.소매
Blogger Link :https://blog.naver.com/newhasu1
Post Date :20190217
-------------------------------------------------
#921
Title :양산크레인-박대에 대해서 알아볼까요?
Link :https://blog.naver.com/gn-crane?Redirect=Log&logNo=221385044379
Description :오늘은 박대라는 생선에 대해서 알아볼까합니다. 10월에서 12월이 제철인 생선이라네요~ 참 생소한

-------------------------------------------------
#941
Title :참박대 대사이즈(130g이상 사이즈) 할인 계획
Link :https://blog.naver.com/newhasu1?Redirect=Log&logNo=221419238463
Description :참박대 대사이즈 블로그 이웃늘이기 전략 노마진 행사합니다. 많은 물량은 아니고 각 사이즈 30... 참박대 원가 자체가 서대에 비해 워낙 쎄다보니 노마진이어도 비싸게 보이는건 마찬가지일... 
Blogger Name :군산건어물(행남수산) 도.소매
Blogger Link :https://blog.naver.com/newhasu1
Post Date :20181214
-------------------------------------------------
#942
Title :군산 박대구이
Link :https://blog.naver.com/tsh989?Redirect=Log&logNo=221209001791
Description :오늘은 군산 박대구이 에 대한 리뷰입니닷. 저는 시골이 강원도쪽이라 개인적으로 여행가는거아니면... 근데 진짜 좋더라구요 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 그리고 그쪽 어시장에서 구경좀하다가 사온 박대 리뷰를..... 
Blogger Name :Hun's ordinary day
Blogger Link :https://blog.naver.com/tsh989
Post Date :20180216
-------------------------------------------------
#943
Title :2015-06-19 : 해루질/서해/태안/해루질/소라,광어,도다리,박대... 
Link :https://blog.naver.com/dhjssmjm?Redirect=Log&logNo=220395905238
Description :흐미~~~이거 감이 죽이네요...^^ 이후 그이도 그이지만 광순이,도돌이 찾아서 요리조리 돌아다녀서 도다리,광어,박대,주먹이,골뱅

-------------------------------------------------
#963
Title :반건조'박대'구이.
Link :https://blog.naver.com/k4231212?Redirect=Log&logNo=70105075342
Description :박대 후라이팬보다 박대가 더 커서 꼬리까지 다 들어가질 않네요.ㅠ 머리부터 구운 후 꼬리 넣어 구웠어요. 나중엔 반잘라서 구워야겠네요. 한마리구워서 이쁘게 찍을려고 했건만...... 박대 구입후 랩에 싸서... 
Blogger Name :☆토끼야 놀자☆
Blogger Link :https://blog.naver.com/k4231212
Post Date :20110317
-------------------------------------------------
#964
Title :서대(박대)조림
Link :https://blog.naver.com/nailstan?Redirect=Log&logNo=150070487333
Description :서대(박대)조림 요리재료 반건조 서대 2마리, 무 중간크기 4분의 1토막, 양파 작은것 1개, 멸치다시육수 반컵 대파 혹은 잔파 한주먹, 홍고추, 청고추 1개씩 양념 ; 외간장, 집간장, 2큰술씩, 청주 3큰술, 마늘... 
Blogger Name :행복물결 넘실대는 여행이야기
Blogger Link :https://blog.naver.com/nailstan
Post Date :20090924
-------------------------------------------------
#965
Title :군산맛집 아리랑 황금박대정식 맛있네요.
Link :https://blog.naver.com/whiffle?Redirect=Log&logNo=221207267546
Description :난 황금박대정식이 먹고 싶었다. 가격은 2만원.. 주문했다. 박대구이다... 어릴적부터 참 좋아했던 박대구이. 것도 황금박대구이란다. 색감이 살짝 금빛을 띄긴 한건 같다.

-------------------------------------------------
#985
Title :마마무 화사 박대 구이 '이것'에 중독
Link :https://blog.naver.com/hanous?Redirect=Log&logNo=221326976358
Description :마마무의 화사는 25일 방송된 JTBC '한끼줍쇼' 에서 박대 구이 먹방을 선보인다. 결국, 화사는 뭘 먹어도 화제가 됩니다. 이날 화사는 밥동무 어머니가 해주신 박대 구이 홀릭에 빠진 모습을 보여줍니다. 박대를... 
Blogger Name :바싹한 킷캣
Blogger Link :https://blog.naver.com/hanous
Post Date :20180726
-------------------------------------------------
#986
Title :스타벅스 요거트자, 파리바게트 타르트, 박대구이, 산성동맛집... 
Link :https://blog.naver.com/landbeen?Redirect=Log&logNo=221452792102
Description :사실 전 딸기에 설탕 잔뜩 뿌려서 후루룩 마시는파예요 ㅋㅋ 가끔 생크림도 찍어먹지만 역시 설탕에 빠진 딸기가 꿀맛이지요 헤헷 생선을 잔뜩 구입해서 소분해두웠어요 박대랑 조기를 샀는데 박대 살짝... 
Blogger Name :동남공인중개사사무소
Blogger Link :https://blog.naver.com/landbeen
Post Date :20190128
-------------------------------------------------
#987
Title :[수미네 반찬] 반건조 박대구이와 박대조림 황금 레시피! 누구나... 
Link :https://blog.naver.com/leehyewon84?Redirect=Log&logNo=221386080587
Description :지난 여름, 화사가 출연한 ‘한끼줍쇼’를 통해 장안에 화제가 되었던 '박대'가 tvN ‘수미네 

## 형태소 분석

- mecab : window에서 안되는 걸로 알고 있음.
- 한나눔 : 성능이 떨어지는 것으로 알고 있음
- 카이(카카오 형태소 분석기) : 사전 추가가 안됨.
- 트위터 분석기, 코모란 두 가지를 중점.

In [6]:
from konlpy.tag import *
import time

In [7]:
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor

twitter = Twitter()
okt_nouns = Postprocessor(twitter, passtags={'Noun'})

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [8]:
query='박대'

In [9]:
# 데이터 입수
text = open(query+'.txt',encoding='utf-8').read() 

In [10]:
okt_result=okt_nouns.pos(text)

## 추가적인 불용어(stopword) 제거


In [40]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [41]:
stopwords=pd.read_csv("한글불용어.csv",encoding='utf-8')
stopword=stopwords["불용어"].tolist()

# 명사 카운팅 Top100

In [42]:
raw_data=pd.DataFrame(okt_result)
nostop_data=raw_data

In [43]:
for word in stopword :
    nostop_data=nostop_data[nostop_data[0]!=word]

In [37]:
from collections import Counter
import pandas as pd
import numpy as np

In [50]:
nostop_data_list=nostop_data[0].tolist()
count=Counter(nostop_data_list)
counting_data=pd.DataFrame(count.most_common(100))
counting_data.to_csv(query+"_카운팅.csv",encoding="ms949")
counting_data.head(10)

,0,1
0,박대,8357
1,생선,3050
2,맛,2177
3,어요,1929
4,군산,1724
5,구이,1570
6,요,1247
7,조림,1197
8,어서,979
9,마리,907


## 워드 클라우드 생략

# Word2Vec 적용

- 내가 원하는 키워드로 크롤링한 모든 데이터를 학습시켜 만든 모델.

In [5]:
from konlpy.tag import Okt 
import pandas as pd

1) 크롤링된 데이터를 포스트 단위로 리스트 생성

- 이후의 오류 방지를 위해 비어있는 포스트의 경우에는 제거한다.

In [15]:
post_list=text.split("-----------------------------------")
print(len(post_list))
for i in range(len(post_list)) :
    if(len(post_list[i]) == 0):
        print(i)
        del post_list[i]
        
print(len(post_list))        

1070
1069
1069


2) 각각의 포스트에 대하여 형태소 분석기를 돌려서 명사만을 저장 [ ( '단어' , '품사' ) , ...... ]

In [16]:
post_list1=[]

for post1 in post_list :
    okt_post=okt_nouns.pos(post1)
    post_list1.append(pd.DataFrame(okt_post))

3) 저장된 명사 리스트[ ( '단어' , '품사' ) , ...... ]를 데이터프레임화 시킨 후 품사부분을 제거

In [17]:
post_list2=[]
for item1 in post_list1 :
    item2=pd.DataFrame(item1)
    post_list2.append(item2)

In [22]:
post_list3=[]
for df in post_list2 :
    df2=pd.DataFrame(df[0])
    post_list3.append(df2)

4) 불용어 제거 ( 산출물 :DataFrame -> List )

In [26]:
post_list4=[]
for df in post_list3 :
    nonstop_df=df
    for word in stopword :
        nonstop_df=nonstop_df[nonstop_df[0]!=word]
    post_list4.append(nonstop_df)    

In [30]:
post_list5=[]
for df in post_list4 :
    df_to_list=df[0].values.tolist()
    post_list5.append(df_to_list)

5) Word2Vec 적용

In [34]:
from gensim.models import Word2Vec
model = Word2Vec(post_list5, size=100, window=5, min_count=5, workers=4, sg=0)

In [35]:
friends=model.wv.most_similar(query)
finish=pd.DataFrame(friends)
finish.to_csv(query+"word2vec.csv")
finish

,0,1
0,종종,0.936352
1,요리,0.931227
2,미네,0.921317
3,두둥,0.919748
4,대조,0.912538
5,리해,0.910852
6,구이,0.907459
7,요,0.902463
8,러시,0.886372
9,영감,0.886337
